# Responsible ML: Credit Risk Interpretability with LIME and SHAP

This notebook demonstrates how to explain credit-risk model predictions using **LIME** (local, model-agnostic explanations) and **SHAP** (Shapley-based additive explanations). The goal is to make individual decisions auditable and easier to communicate to stakeholders.

Dataset: **German Credit** (via AIF360).

## Why interpretability matters

For credit risk, it is not enough to output a class label (e.g., *good* vs *bad*). Practitioners often need to answer:

- **Why** did the model make this prediction for this applicant?
- Which features **most influenced** the decision?
- Are the explanations **stable** and **consistent** across methods?

This notebook compares **LIME** and **SHAP** on the same trained model to produce instance-level explanations and feature attributions.

![Local explanation example (LIME)](https://raw.githubusercontent.com/marcotcr/lime/master/doc/images/lime.png)

In [ ]:
#@title Install LIME
%%capture
!pip install lime

In [ ]:
#@title Install AIF360 utilities
%%capture
!pip install aif360['all']

In [ ]:
#@title Import libraries


# core

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

# sklearn

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.datasets import load_breast_cancer


# Load data from AIF360

from aif360.sklearn.datasets import fetch_german



## Data acquisition and preparation

We use the **German Credit** dataset from **AIF360**, which includes both predictive features and protected attributes commonly discussed in fairness and responsible ML contexts.

In [ ]:
# Load the data.

X, y = fetch_german()
X.head()

In [ ]:
# Convert to a standard modeling format.

df = pd.concat([X,y], axis=1).drop(columns=['sex',  'foreign_worker']).rename(columns={'age': 'age_numeric'}).reset_index()
df.head()

In [ ]:
df = df.rename(columns={'age': 'age_cat', 'age_numeric': 'age'})
df.info()

## Feature dictionary (German Credit)

**Numeric features**
- `duration`: Loan duration in months (4–72)
- `credit_amount`: Requested credit amount (250–18424, in Deutsche Mark)
- `installment_commitment`: Installment rate as % of disposable income (1–4)
- `residence_since`: Years at current residence (1–4)
- `age`: Age in years (19–75)
- `existing_credits`: Number of existing credits at this bank (1–4)
- `num_dependents`: Number of dependents (1–2)

**Categorical features (examples)**
- `checking_status`, `credit_history`, `purpose`, `savings_status`, `employment`, `housing`, `job`, etc.
- Protected attributes commonly used in analyses: `sex`, `foreign_worker`

**Target**
- `credit-risk`: `'good'` (favorable) or `'bad'` (unfavorable)

## Preprocessing

We encode categorical variables, split into train/test sets, and fit a baseline classifier suitable for demonstrating explanation methods. The focus is on **explainability**, not on extensive model tuning.

In [ ]:
df.info()

In [ ]:
# Binarize selected variables.

df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['age_cat'] = df['age_cat'].map({'aged': 1, 'young': 0})
df['foreign_worker'] = df['foreign_worker'].map({'no': 1, 'yes': 0})
df['credit-risk'] = df['credit-risk'].map({'good': 1, 'bad': 0})

In [ ]:
# Separamos X e Y.

X = df.loc[:, df.columns != 'credit-risk']
y = df.loc[:, df.columns == 'credit-risk']

In [ ]:
# Necesitamos ocupar Label Encoder
to_encode = X.select_dtypes(include='category')
encoders = {}
for catcol in to_encode.columns:
  encoder = LabelEncoder()
  X[catcol] = encoder.fit_transform(X[catcol])
  encoders[catcol] = encoder

X.head()

In [ ]:
# Train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=908)

In [ ]:
# Exclude age feature

X_train.drop(columns=['age'], inplace=True)
X_test.drop(columns=['age'], inplace=True)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, np.ravel(y_train))

In [ ]:
y_pred = rf.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
categorical_features = [X_train.columns.get_loc(col) for col in to_encode.columns]
categorical_names = {i: encoders[X_train.columns[i]].classes_ for i in categorical_features}

In [ ]:
categorical_features

In [ ]:
categorical_names

In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(X_train.values,
                                 mode='classification',
                                 feature_names=X_train.columns.to_list(),
                                 categorical_features=categorical_features,
                                 categorical_names=categorical_names,
                                 discretize_continuous=True,
                                 discretizer='decile',
                                 kernel_width=5)

In [ ]:
exp = explainer.explain_instance(X_train.iloc[3,:], rf.predict_proba, num_features=5)

In [ ]:
exp.as_pyplot_figure()
plt.show()

In [ ]:
exp.show_in_notebook(show_all=False)

In [ ]:
pd.DataFrame(exp.as_list(),columns=['Feature','Contribution'])

# SHAP: Global and local attributions

## SHAP overview

**SHAP** explains predictions using Shapley-value principles from cooperative game theory. It provides:

- **Local explanations**: feature contributions for a single prediction
- **Global insights**: aggregated feature importance across many predictions

Compared with LIME, SHAP often yields more consistent attributions for tree-based models and can be used to summarize behavior across the dataset.

In [ ]:
%%capture
!pip install shap

In [ ]:
import shap

In [ ]:
# Continuing with the German Credit dataset workflow.
X_train.head()

In [ ]:
# Train a Random Forest model
rf

In [ ]:
# Create an explainer and compute SHAP values
explainer = shap.Explainer(rf.predict_proba, X_train)
shap_values = explainer(X_train[:100])

In [ ]:
# Keep SHAP values for the positive class.
shap_values = shap_values[..., 1]

In [ ]:
# Bar plot
shap.plots.bar(shap_values)

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.scatter(shap_values[:,"checking_status"])

In [ ]:
shap.plots.scatter(shap_values[:,"duration"])

In [ ]:
shap.plots.waterfall(shap_values[99])